<a href="https://colab.research.google.com/github/Eimaen/yet-another-waifu-generator/blob/main/waifu_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Install libraries

%pip install --quiet --upgrade cython git+https://github.com/huggingface/diffusers transformers accelerate mediapy scipy safetensors xformers triton discord-webhook opencv-python

In [ ]:
#@title Install packages

!apt install aria2

In [ ]:
#@title Import Real-CUGAN Upscaler

!git clone https://github.com/bilibili/ailab.git ./upscaler

!aria2c -x 8 https://eimaen.pw/files/RealCUGAN.zip -o upscaler/Real-CUGAN/model/weights.zip
!unzip -o upscaler/Real-CUGAN/model/weights.zip -d upscaler/Real-CUGAN/model
!rm upscaler/Real-CUGAN/model/weights.zip

import importlib.util
upcunet_v3_spec = importlib.util.spec_from_file_location("RealWaifuUpScaler","upscaler/Real-CUGAN/upcunet_v3.py")
upcunet_v3 = importlib.util.module_from_spec(upcunet_v3_spec)
upcunet_v3_spec.loader.exec_module(upcunet_v3)

In [ ]:
#@title Download model

!rm -rf model
!mkdir model
!aria2c -x 8 https://eimaen.pw/files/anything-v3.zip -o model/model.zip
!unzip -o model/model.zip -d model
!rm model/model.zip

In [ ]:
#@title Import libraries

from diffusers import StableDiffusionPipeline
import torch
import mediapy as media
from PIL import Image
from io import BytesIO
from discord_webhook import DiscordWebhook, DiscordEmbed
import cv2

In [ ]:
#@title Load the image generation pipeline (txt2img)

pipe = StableDiffusionPipeline.from_pretrained("model", torch_dtype=torch.float16, safety_checker=None)
pipe = pipe.to("cuda")
pipe.enable_xformers_memory_efficient_attention()

In [ ]:
#@title Generate an image { display-mode: "form" }

#@markdown ## Prompt:
prompt = "konpaku youmu, :<, 1girl, standing, green skirt, green vest, shirt tucked in, hair ribbon, hairband, arms behind back, collared shirt, small breasts, white hair, short hair" #@param {type:"string"} 
negative_prompt = "multiple breasts, (mutated hands and fingers:1.5), (long body:1.3), mutation, black-white, bad anatomy, liquid body, liquid tongue, disfigured, malformed, mutated, anatomical nonsense, text font ui, error, malformed hands, long neck, blurred, lowers, lowres, bad anatomy, bad proportions, bad shadow, uncoordinated body, unnatural body, fused breasts, bad breasts, huge breasts, poorly drawn breasts, extra breasts, liquid breasts, \u200Bheav\u200By breasts, missing breasts, huge haunch, huge thighs, huge calf, bad hands, fused hand, missing hand, disappearing arms, disappearing thigh, disappearing calf, disappearing legs, fused ears, bad ears, poorly drawn ears, extra ears, liquid ears, heavy ears, missing ears, fused animal ears, bad animal ears, poorly drawn animal ears, extra animal ears, liquid animal ears, heavy animal ears, missing animal ears, text, ui, error, missing fingers" #@param {type:"string"}

#@markdown ---
#@markdown ## Image resolution:
height = 512 #@param {type:"slider", min:64, max:1024, step:8}
width = 512 #@param {type:"slider", min:64, max:1024, step:8}

#@markdown ---
#@markdown ## AI tweaks:
interference_steps = 64 #@param {type:"slider", min:24, max:256, step:1}
guidance_scale = 11.2 #@param {type:"slider", min:1, max:32, step:0.1}
amount_to_generate = 1 #@param {type:"slider", min:1, max:10, step:1}

print("Generating...")
images = pipe(prompt, height = height, width = width, num_inference_steps = interference_steps, guidance_scale = guidance_scale, num_images_per_prompt = amount_to_generate, negative_prompt = negative_prompt).images

#@markdown ---
#@markdown ## Upscaling:
upscale_images = False #@param {type:"boolean"}
upscaler_model = "up2x-latest-denoise1x.pth" #@param ["up2x-latest-conservative.pth", "up2x-latest-denoise1x.pth", "up2x-latest-denoise2x.pth", "up2x-latest-denoise3x.pth", "up2x-latest-no-denoise.pth"]

if upscale_images:
    print("Upscaling...")
    upscaler = upcunet_v3.RealWaifuUpScaler(2, "upscaler/Real-CUGAN/model/" + upscaler_model, half=True, device="cuda:0")
    images = map(lambda image: Image.fromarray(upscaler(image,tile_mode=5,cache_mode=2,alpha=1)), images)

media.show_images(images)

#@markdown ---
#@markdown ## Saving data:
save_data = False #@param {type:"boolean"}
webhook_url = "" #@param {type:"string"}
add_description = False #@param {type:"boolean"}

if save_data:
    for image in images:
        webhook = DiscordWebhook(url=webhook_url)

        if add_description:
            embed = DiscordEmbed(title="Model Output", description="**Prompt:**\n```{}```\n**Negative prompt:**\n```{}```\n**Interference steps:** `{}`\n**Guidance scale:** `{}`\n\n**Width:** `{}`\n**Height:** `{}`".format(prompt, negative_prompt, interference_steps, guidance_scale, width, height), color="03b2f8")
            embed.set_author(name="Stable Diffusion", url="https://eimaen.pw/files/anything-v3.zip", icon_url="https://cdn.discordapp.com/attachments/893438518066556938/1082396674137268276/output.png")
            webhook.add_embed(embed)

        byte_array = BytesIO()
        image.save(byte_array, format="PNG")
        webhook.add_file(file=byte_array.getvalue(), filename="output.png")

        webhook.execute()
